In [5]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import kagglehub
import os

path = kagglehub.dataset_download("chenxaoyu/modelnet-normal-resampled")

print("Path to dataset files:", path)

dataset_root = ""
for root, dirs, files in os.walk(path):
    if 'airplane' in dirs:
        dataset_root = root
        break

print(f"Verified Dataset Root: {dataset_root}")

100%|██████████| 1.65G/1.65G [00:20<00:00, 88.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/chenxaoyu/modelnet-normal-resampled/versions/1
Verified Dataset Root: /root/.cache/kagglehub/datasets/chenxaoyu/modelnet-normal-resampled/versions/1/modelnet40_normal_resampled


In [2]:
!git clone https://github.com/BardiaAkbari/Final-CV-Project.git

Cloning into 'Final-CV-Project'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 39 (delta 6), reused 38 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 16.03 MiB | 41.36 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [6]:
import os
import sys
import torch
import gc

# 1. CLEANUP MEMORY (Crucial Step!)
# We need to free the VRAM held by the crashed process
torch.cuda.empty_cache()
gc.collect()
print("🧹 GPU Memory cleared.")

# 2. Ensure we are in the correct directory
os.chdir('/content/Final-CV-Project/Code')

# 3. SAFETY PATCH (Ensures model.py is still correct)
# (Just in case the file got reverted or changed)
model_path = 'models/model.py'
if os.path.exists(model_path):
    with open(model_path, 'r') as f:
        code = f.read()
    if "import numpy as np" not in code:
        code = "import numpy as np\n" + code
    if ".view(B*N_new*K, C)" in code:
        code = code.replace(".view(B*N_new*K, C)", ".reshape(B*N_new*K, C)")
    with open(model_path, 'w') as f:
        f.write(code)

# 4. RUN TRAINING WITH LOWER BATCH SIZE
models = ['pointtransformer38', 'pointtransformer50']

# Ensure dataset root is defined (fallback check)
if 'dataset_root' not in locals():
    # Attempt to locate it again if the variable was lost in a restart
    possible_root = "/root/.cache/kagglehub/datasets/chenxaoyu/modelnet-normal-resampled/versions/1"
    if os.path.exists(possible_root):
        dataset_root = possible_root
    else:
        # Last resort: check local link
        dataset_root = os.path.abspath("data/modelnet40_normal_resampled")

for model_name in models:
    print(f"\n{'='*40}")
    print(f"🚀 STARTING TRAINING FOR: {model_name}")
    print(f"{'='*40}\n")

    # Check data link
    !mkdir -p data
    !ln -sfn "{dataset_root}" data/modelnet40_normal_resampled

    # COMMAND CHANGE: batch_size 16 -> 8
    command = (
        f"python train.py "
        f"--model {model_name} "
        f"--batch_size 8 "       # <--- REDUCED FROM 16 TO 8
        f"--epoch 150 "
        f"--lr 1e-3 "
        f"--checkpoint_dir ./checkpoints_{model_name} "
    )

    !{command}

    print(f"\n✅ FINISHED TRAINING: {model_name}")

    # Clear memory again between models
    torch.cuda.empty_cache()z

SyntaxError: invalid syntax (ipython-input-1568635881.py, line 65)